In [ ]:
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
import pandas as pd
import torch
import yaml
from tqdm import tqdm

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch import multiprocessing
from torchsummary import summary


from mars_model import BinaryClassifier
from mars_dataset import MARSDataset

%matplotlib inline

In [ ]:
torch.cuda.empty_cache()
torch.set_default_tensor_type('torch.cuda.FloatTensor')
multiprocessing.set_start_method('spawn')

In [ ]:
with open('detector_config.yaml', 'r') as ymlfile:
    config = yaml.load(ymlfile, Loader=yaml.Loader)

In [ ]:
data = pd.read_json(config['DATASET_ROOT'] + config['DATASET_JSON'])
data.dropna(inplace=True)
# data['y'] = data['label'].apply(lambda x: 1 if x != 'no_whale' else 0) # baseline
data['y'] = data['label'].apply(lambda x: 1 if x == 'whale+' else 0) # highSNR

Balancing dataset

In [ ]:
num_whale = data[data['y'] == 1]['y'].count() # there are many fewer whale samples than no_whale
whale = data[data['y'] == 1]
nowhale = data[data['y'] == 0].sample(n=num_whale, random_state=1)
data = pd.concat([whale, nowhale])

In [ ]:
X_train, X_val = train_test_split(data, 
                                  test_size=config['TEST_SIZE'], 
                                  random_state=config['RANDOM_STATE'],
                                  stratify=data['y'],
                                  )

trainset = MARSDataset(X_train)
testset = MARSDataset(X_val)

from tutorial: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [ ]:
trainloader = DataLoader(trainset,
                         batch_size=config['N_BATCH'], 
                         shuffle=True, 
                         num_workers=2,
                         generator=torch.Generator(device='cuda'))

testloader = DataLoader(testset, 
                        batch_size=config['N_BATCH'], 
                        shuffle=True, 
                        num_workers=2,
                        generator=torch.Generator(device='cuda'))

classes = (0, 1)

In [ ]:
net = BinaryClassifier().cuda()

criterion = nn.BCELoss()
# optimizer = optim.SGD(net.parameters(), 
#                       lr=config['LEARNING_RATE'], 
#                       momentum=config['MOMENTUM'])
optimizer = optim.Adam(net.parameters(),
                       lr=config['LEARNING_RATE'])

summary(net, (1, 180, 1244))

In [ ]:
# net = SimpleCNN()
# net.cuda()

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), 
#                       lr=config['LEARNING_RATE'], 
#                       momentum=config['MOMENTUM'])



In [ ]:
# net = ResNet(ResidualBlock, [1, 4, 6, 3], 2).cuda()
# criterion = nn.CrossEntropyLoss()
# # optimizer = optim.SGD(net.parameters(), 
# #                       lr=config['LEARNING_RATE'], 
# #                       momentum=config['MOMENTUM'])

# optimizer = optim.Adam(net.parameters(),
#                        lr=config['LEARNING_RATE'])

In [ ]:
min_loss = 1000.0 # initialize to large value

In [ ]:
for epoch in range(config['N_EPOCHS']): # loop through epochs
    print(f'Epoch: {epoch+1} / {config["N_EPOCHS"]}')
    running_loss = 0.0 # reset
    
    for i, (inputs, labels, _) in enumerate(tqdm(trainloader), 0): # loop through batches
        
        # move data to GPU
        # inputs = inputs.unsqueeze(1).to(torch.device('cuda'))     
        inputs = inputs.cuda()  
        labels = labels.cuda().float()
        
        # check that inputs are valid
        assert ~torch.isnan(inputs).any(), f'input error (nan): {inputs}'
        assert ~torch.isinf(inputs).any(), f'input error (inf): {inputs}'
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # generate statistics
        running_loss += loss.item()
    epoch_loss = running_loss / (i+1)
    print(f'Loss: {epoch_loss:.4f}, min loss: {min_loss}')
    if epoch_loss < min_loss:
        min_loss = epoch_loss
        print('saving...')
        torch.save(net.state_dict(), config['MODEL_ROOT'] + config['MODEL_NAME'] + '_b' + '.pth')

print('Finished Training')

In [ ]:
net = BinaryClassifier().cuda()
# net = ResNet(ResidualBlock, [1, 4, 6, 3], 2).cuda()
# net = SimpleCNN().cuda()

net.load_state_dict(torch.load(config['MODEL_ROOT'] + config['MODEL_NAME'] + '.pth'))

In [ ]:
y_true = []
y_pred = []

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for i, (inputs, labels, _) in enumerate(tqdm(testloader)):
        inputs = inputs.cuda()
        labels = labels.cuda().float()
        # calculate outputs by running images through the network
        outputs = net(inputs)
        # the class with the highest energy is what we choose as prediction
        # _, predicted = torch.max(outputs.data, 1) # for softmax classifier
        predicted = outputs.round() # for binary classifier with sigmoid
        y_true += labels.cpu().tolist()
        y_pred += predicted.cpu().tolist()
try:
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
except:
    pass

# print(f'Accuracy of the network on the testset: {(correct / total):.4f}')
print(f'Accuracy:  {accuracy_score(y_true, y_pred):.4f}')
print(f'Precision: {precision_score(y_true, y_pred):.4f}')
print(f'Recall:    {recall_score(y_true, y_pred):.4f}')
print(f'F1:        {f1_score(y_true, y_pred):.4f}')
print(f'Confusion:\nTP: {tp}\nFP: {fp}\nTN: {tn}\nFN: {fn}')

model using BinaryClassifier on Mel-spectrogram data

Accuracy:  0.8696
- Precision: 0.8806
- Recall:    0.8551
- F1:        0.8676
- Confusion:
    - TP: 59
    - FP: 8
    - TN: 61
    - FN: 10

examine errors and update data labels as needed

In [ ]:
# def change_label(filename, label):
#     df = pd.read_json(config['DATASET_ROOT'] + config['DATASET_JSON'])
#     df.loc[df['filename']== filename, 'label'] = label
#     df.to_json(config['DATASET_ROOT'] + config['DATASET_JSON'])

In [ ]:
# change_label('20230923_100023Z.mp3', 'whale')

In [ ]:
# change_label('20231002_010052Z.mp3', 'whale')


In [ ]:
# # since we're not training, we don't need to calculate the gradients for our outputs
# with torch.no_grad():
#     for i, (inputs, labels, fname) in enumerate(tqdm(testloader)):
#         inputs = inputs.unsqueeze(1).to(torch.device('cuda'))
#         labels = labels.to(torch.device('cuda'))
#         # calculate outputs by running images through the network
#         outputs = net(inputs)
#         # the class with the highest energy is what we choose as prediction
#         _, predicted = torch.max(outputs.data, 1)
#         y_true = labels.cpu().tolist()
#         y_pred = predicted.cpu().tolist()
#         if (y_true==[1]) and (y_pred==[0]):
#             print(f'{fname}\npredicted: {y_pred}\nground truth: {y_true}')
#             plt.imshow(np.flipud(inputs.cpu().numpy().squeeze()), cmap='jet')
#             plt.show()


In [ ]:
# from mars_clip import MarsClip
# clip = MarsClip('20231005_040052Z.mp3')

In [ ]:
# sxx, _, _ = clip.get_spec_img()
# plt.imshow(np.flipud(sxx), cmap='jet')
# plt.show()